In [1]:
import os

In [3]:
print(os.getcwd())
os.chdir("../")
print(os.getcwd())

c:\Users\A2Z\Desktop\MLOPs\cicd-project
c:\Users\A2Z\Desktop\MLOPs


In [6]:
from dataclasses import dataclass 
from pathlib import Path

In [9]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir : Path 
    test_data_path: Path 
    model_name: Path 
    all_params: dict 
    matric_file_path: Path
    target_column: str

In [8]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN


        create_directories([config.root_dir])
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir, 
            test_data_path=config.test_data_path, 
            model_name=config.model_name, 
            all_params=params, 
            matric_file_path=config.metric_file_path, 
            target_column=schema.name
        )

        return model_evaluation_config


In [12]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import numpy as np
import joblib

In [13]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config



    def evaluation_metrics(self, actual, predicted):
        mse = mean_squared_error(actual, predicted)
        mae = mean_absolute_error(actual, predicted)
        r2 = r2_score(actual, predicted)

        return mse, mae, r2 
    
    

    def save_results(self):
        test = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_name)

        test_x = test.drop(columns=[self.config.target_column], axis=1)
        test_y = test[[self.config.target_column]]

        predictions = model.predict(test_x)
        mse, mae, r2 = self.evaluation_metrics(test_y, predictions)

        scores = {"mse": mse, "mae": mae, "r2": r2}
        save_json(path=Path(self.config.matric_file_path), scores=scores)



In [ ]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e